Цель данного проекта: анализ отзывов, полученных от школьников и студентов после прохождения курсов.
В зависимости от того, сколько времени удастся уделить проекту, планируется либо:
- классифицировать отзывы на позитивные и негативные
- выделить основную информацию из текста

Для сбора данных был создан телеграм-бот, который спрашивал у пользователя, какой курс он прошел, и записывал ответы в базу данных.

По результатам работы бота был создан небольшой датасет, который в дальнейшем и планируется анализировать. Переведем его в формат pandas DataFrame для удобства

In [1]:
from bot.models import Feedback
import pandas as pd
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

engine = create_engine('sqlite:///feedbacks.db')
with Session(engine) as session:
    df = pd.read_sql(session.query(Feedback).statement, engine)

df.set_index('id', inplace=True)
print(df)

      course                                            content
id                                                             
1     python                    Все прошло просто замечательно!
2     python                                         1000000/10
3     python                                           Курс 🔥🔥🔥
4   networks                                   Курс великолепен
5     python                                               None
6     python  Замечательный курс по программированию на Pyth...
7     python  Я учусь в 9 классе и недавно прошел этот курс ...
8     python  Я родитель и хочу выразить благодарность созда...
9     python  Этот курс по Python стал настоящим открытием д...
10    python  Как ученик 11 класса, я нашел этот курс по Pyt...
11    python                                     /give_feedback
12    python  Омг, этот курс по Питону просто пушка! Училка ...
13    python  Братан, этот курс по Python меня просто улетел...
14    python  Курс по Питону просто топч

Как видно из полученных значений, большая часть из них - короткие фразы, в которых есть:
- Смайлики;
- Знаки препинания for empathise;
- Сленговые слова;
- Слова из IT, часть которых переведена транслитом на русский язык (Python - Питон, Git - гит);
- Опечатки;
- Некоторые мусорные варианты, такие как /give_feedback или None (что предположительно являлось стикером, но не обработалось ботом корректно при записи БД).

Обработаем эти данные

In [2]:
# Почистим пустые комментарии
df.dropna(inplace=True)

# Почистим комментарии с командой /give_feedback, потому что это доступая для бота команда
df = df.loc[df['content'] != '/give_feedback']

# Приведем данные к нижнему регистру
df['lowered_content'] = df['content'].apply(lambda text: text.lower())
print(df['lowered_content'])

id
1                       все прошло просто замечательно!
2                                            1000000/10
3                                              курс 🔥🔥🔥
4                                      курс великолепен
6     замечательный курс по программированию на pyth...
7     я учусь в 9 классе и недавно прошел этот курс ...
8     я родитель и хочу выразить благодарность созда...
9     этот курс по python стал настоящим открытием д...
10    как ученик 11 класса, я нашел этот курс по pyt...
12    омг, этот курс по питону просто пушка! училка ...
13    братан, этот курс по python меня просто улетел...
14    курс по питону просто топчик, ребят! от начала...
15    этот курс по python прям космос! училка объясн...
16    ахах, ну и курс! реально весело и понятно все ...
17                                         клевый курс 
18    все было хорошо, хотя местами хотелось больше ...
19      на курсе было сложно, но рад, что закончил его 
20    сделали классного бота, прикольные прим

установим библиотеку razdel для токенизации предложений


In [5]:
from razdel import tokenize
df['tokenized_content'] = df['lowered_content'].apply(lambda text: [_.text for _ in tokenize(text)])
print(df['tokenized_content'])

id
1                [все, прошло, просто, замечательно, !]
2                                          [1000000/10]
3                                           [курс, 🔥🔥🔥]
4                                   [курс, великолепен]
6     [замечательный, курс, по, программированию, на...
7     [я, учусь, в, 9, классе, и, недавно, прошел, э...
8     [я, родитель, и, хочу, выразить, благодарность...
9     [этот, курс, по, python, стал, настоящим, откр...
10    [как, ученик, 11, класса, ,, я, нашел, этот, к...
12    [омг, ,, этот, курс, по, питону, просто, пушка...
13    [братан, ,, этот, курс, по, python, меня, прос...
14    [курс, по, питону, просто, топчик, ,, ребят, !...
15    [этот, курс, по, python, прям, космос, !, учил...
16    [ахах, ,, ну, и, курс, !, реально, весело, и, ...
17                                       [клевый, курс]
18    [все, было, хорошо, ,, хотя, местами, хотелось...
19    [на, курсе, было, сложно, ,, но, рад, ,, что, ...
20    [сделали, классного, бота, ,, прикольны

Задания дальше находятся в разработке, потому что возникли проблемы:
- Как совершить морфологический анализ, приведя все слова к начальной форме, и при этом оставить некоторые фразы как есть (например, 10/10)
- Как пракильно убрать ненужные знаки препинания
